<h1> Code used for Experiment 223 </h1>

This Python notebook shows exactly what code was used to process the images of the shapes, and find their centres of mass.

Firstly, the following modules were imported:

In [1]:
import matplotlib.pyplot as plt
import numpy as np

# this is to make sure figures pop out as a separate window:
%matplotlib tk 

# loading standard colour maps
from matplotlib import cm 

Next, the images were imported as arrays of RGB values for each pixel. 

In [8]:
F1 = plt.imread(".\\Shape1.jpg")
F2 = plt.imread(".\\Shape2.jpg")
F3 = plt.imread(".\\Shape3.jpg")

To find the centres of mass of each shape, the pixels on the shape had to be isolated from the others. This was done by converting the image to grayscale, and setting a threshold brightness for the pixels that would be considered on the shape. This threshold was found by trial and error.

In [9]:
def threshold_color_image(F, threshold = 0.5):
    
    # Weighting the components to make the image grayscale
    imgray = 0.2125*F[:,:,0] + 0.7154*F[:,:,1] + 0.0721*F[:,:,2]
    
    # Normalizing brightness of the pixels
    imgray = imgray / imgray.max()
    
    # Determining the pixels brighter than the threshold chosen
    mask_bright = imgray > threshold
    mask_dark = np.logical_not(mask_bright)
    
    # Setting each pixel on the shape to be 1, others 0
    imgray[mask_bright] = 1
    imgray[mask_dark] = 0

    return imgray

# Setting image arrays to their indicator function equivalents
# (info about colour/brightness is no longer needed)
F1 = threshold_color_image(F1, threshold = 0.7)
F2 = threshold_color_image(F2, threshold = 0.5)
F3 = threshold_color_image(F3, threshold = 0.6)

Next, to generate the images included in the report (and also to make sure the shapes were roughly correct), a matplotlib figure was generated that shows the shape it thinks the object is.

In [10]:
fig, ax = plt.subplots()
ax.set_title('Reading of Shape 1')
implot = ax.imshow(F1, cmap=cm.gray, aspect='equal')
ax.set_xlabel('horizontal distance (in pixels)')
ax.set_ylabel('vertical distance (in pixels)')
plt.show()

fig, ax = plt.subplots()
ax.set_title('Reading of Shape 2')
implot = ax.imshow(F2, cmap=cm.gray, aspect='equal')
ax.set_xlabel('horizontal distance (in pixels)')
ax.set_ylabel('vertical distance (in pixels)')
plt.show()

fig, ax = plt.subplots()
ax.set_title('Reading of Shape 3')
implot = ax.imshow(F3, cmap=cm.gray, aspect='equal')
ax.set_xlabel('horizontal distance (in pixels)')
ax.set_ylabel('vertical distance (in pixels)')
plt.show()

Based on Equation 18 in the report, the centre of mass was then calculated for each shape.

Firstly, to get a sense of scale, the dpi of each image was calculated. This was done by measuring the distance between two points on each shape (in inches), and then calculating the pixel distance between the two points on the image (using the above figures generated; there is a feature that shows the pixel the mouse is hovering over).

Secondly, a function to find the area of the shape in square inches was defined.

Thirdly, the COM was calculated by summing the $x$-coordinates and $y$-coordinates of each particle in the mass. This is because
\begin{align}
\mathbf{R} = \frac{1}{A} \left(\sum_{i = 1} ^{N_1} \sum_{j = 1} ^{N_2} \chi_Q(\mathbf{r}_{ij})\mathbf{r}_{ij} \, \Delta x \, \Delta y \right)
\end{align}
and $\mathbf{r}_{ij} = x_{ij}\mathbf{\hat{x}} + y_{ij}\mathbf{\hat{y}}$, so
\begin{align}
\mathbf{R} = \frac{1}{A} \left(\sum_{i = 1} ^{N_1} \sum_{j = 1} ^{N_2} \chi_Q(\mathbf{r}_{ij})x_{ij} \, \Delta x \, \Delta y \right)\mathbf{\hat x} + \frac{1}{A} \left(\sum_{i = 1} ^{N_1} \sum_{j = 1} ^{N_2} \chi_Q(\mathbf{r}_{ij})y_{ij} \, \Delta x \, \Delta y \right)\mathbf{\hat y}.
\end{align}

In [11]:
dpi1 = 3171/(208/25.4) # Pixel distance 3171, real distance 208mm
dpi2 = 3339/(216/25.4) # Pixel distance 3339, real distance 216mm
dpi3 = 3197/(158/25.4) # Pixel distance 3197, real distance 158mm

# The area in square dots should just be the number of pixels.
def areainch(img, dpi):
    return np.sum(img)/dpi**2

# This is the answer to [Q7].
def area(img, dpi):
    return np.sum(img)/(dpi/25.4)**2

A1 = areainch(F1, dpi1)
A2 = areainch(F2, dpi2)
A3 = areainch(F3, dpi3)
print(f'The area of shape 1 is {A1:.4g} sq in.')
print(f'The area of shape 2 is {A2:.4g} sq in.')
print(f'The area of shape 3 is {A3:.4g} sq in.')

The area of shape 1 is 29.72 sq in.
The area of shape 2 is 50.42 sq in.
The area of shape 3 is 26.72 sq in.


In [12]:
def cofm(img, dpi):
    x = np.arange(img.shape[1])
    y = np.arange(img.shape[0])
    
    xcm = np.sum(np.transpose(img)*x[:, np.newaxis])/(dpi**2*areainch(img, dpi))
    ycm = np.sum(img*y[:, np.newaxis])/(dpi**2*areainch(img, dpi))
    
    return np.array([xcm, ycm]) # [Q8]

x1 = cofm(F1, dpi1)
x2 = cofm(F2, dpi2)
x3 = cofm(F3, dpi3)
print(f'The COM of shape 1 is at {x1}.')
print(f'The COM of shape 2 is at {x2}.')
print(f'The COM of shape 3 is at {x3}.')

The COM of shape 1 is at [1694.11468402 2002.85540058].
The COM of shape 2 is at [1798.41484467 2012.41727589].
The COM of shape 3 is at [1634.20433664 2184.24639564].


Lastly, the distances from the COM to each hole was calculated. This was done via iteration of the following three code cells: each cell brings up an image of the shape for which clicked pixels are recorded, and calculates the distance between two clicked pixels. This was done by first clicking the COM, and then each hole.

Shape 1:

In [13]:
fig, ax = plt.subplots()
plt.imshow(F1, cmap=cm.gray, aspect='equal')
plt.plot(x1[0], x1[1], 'ro') 

points = plt.ginput(n=5)
plt.show()

# [Q10] (divide by dpi*25.4)
# [Q11]
dist1 = np.linalg.norm(np.array(points[1]) - np.array(points[0]))/dpi1*25.4
dist2 = np.linalg.norm(np.array(points[2]) - np.array(points[0]))/dpi1*25.4
dist3 = np.linalg.norm(np.array(points[3]) - np.array(points[0]))/dpi1*25.4
dist4 = np.linalg.norm(np.array(points[4]) - np.array(points[0]))/dpi1*25.4
print(f'The distance from the COM to hole 1 is {dist1:.4g}.')
print(f'The distance from the COM to hole 2 is {dist2:.4g}.')
print(f'The distance from the COM to hole 3 is {dist3:.4g}.')
print(f'The distance from the COM to hole 4 is {dist4:.4g}.')

The distance from the COM to hole 1 is 105.7.
The distance from the COM to hole 2 is 59.81.
The distance from the COM to hole 3 is 63.43.
The distance from the COM to hole 4 is 102.9.


Shape 2:

In [14]:
fig, ax = plt.subplots()
plt.imshow(F2, cmap=cm.gray, aspect='equal')
plt.plot(x2[0], x2[1], 'ro') 

points = plt.ginput(n=6)
plt.show()
dist1 = np.linalg.norm(np.array(points[1]) - np.array(points[0]))/dpi2*25.4
dist2 = np.linalg.norm(np.array(points[2]) - np.array(points[0]))/dpi2*25.4
dist3 = np.linalg.norm(np.array(points[3]) - np.array(points[0]))/dpi2*25.4
dist4 = np.linalg.norm(np.array(points[4]) - np.array(points[0]))/dpi2*25.4
dist5 = np.linalg.norm(np.array(points[5]) - np.array(points[0]))/dpi2*25.4
print(f'The distance from the COM to hole 1 is {dist1:.4g}.')
print(f'The distance from the COM to hole 2 is {dist2:.4g}.')
print(f'The distance from the COM to hole 3 is {dist3:.4g}.')
print(f'The distance from the COM to hole 4 is {dist4:.4g}.')
print(f'The distance from the COM to hole 5 is {dist5:.4g}.')

The distance from the COM to hole 1 is 77.58.
The distance from the COM to hole 2 is 109.3.
The distance from the COM to hole 3 is 82.29.
The distance from the COM to hole 4 is 112.6.
The distance from the COM to hole 5 is 76.91.


Shape 3:

In [15]:
fig, ax = plt.subplots()
plt.imshow(F3, cmap=cm.gray, aspect='equal')
plt.plot(x3[0], x3[1], 'ro') 

points = plt.ginput(n=6)
plt.show()
dist1 = np.linalg.norm(np.array(points[1]) - np.array(points[0]))/dpi3*25.4
dist2 = np.linalg.norm(np.array(points[2]) - np.array(points[0]))/dpi3*25.4
dist3 = np.linalg.norm(np.array(points[3]) - np.array(points[0]))/dpi3*25.4
dist4 = np.linalg.norm(np.array(points[4]) - np.array(points[0]))/dpi3*25.4
dist5 = np.linalg.norm(np.array(points[5]) - np.array(points[0]))/dpi3*25.4
print(f'The distance from the COM to hole 1 is {dist1:.4g}.')
print(f'The distance from the COM to hole 2 is {dist2:.4g}.')
print(f'The distance from the COM to hole 3 is {dist3:.4g}.')
print(f'The distance from the COM to hole 4 is {dist4:.4g}.')
print(f'The distance from the COM to hole 5 is {dist5:.4g}.')

The distance from the COM to hole 1 is 91.39.
The distance from the COM to hole 2 is 66.2.
The distance from the COM to hole 3 is 81.41.
The distance from the COM to hole 4 is 78.34.
The distance from the COM to hole 5 is 50.25.
